# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = "../WeatherPy/output/cities.csv"
cityDF = pd.read_csv(cities)

cityDF.drop("Unnamed: 0", axis=1,inplace =True)
cityDF

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bethel,41.37,-73.41,298.15,60,20,3.10,US,1594919421
1,Monte Patria,-30.69,-70.95,296.29,19,2,2.12,CL,1594919620
2,Chesma,53.81,60.65,295.15,64,0,2.69,RU,1594919620
3,Oriximiná,-1.77,-55.87,306.35,46,0,3.06,BR,1594919620
4,Saskylakh,71.92,114.08,282.98,58,100,9.75,RU,1594919620
...,...,...,...,...,...,...,...,...,...
574,Clonakilty,51.62,-8.87,295.15,53,40,5.70,IE,1594919691
575,Turukhansk,65.82,87.98,281.97,84,72,6.67,RU,1594919691
576,Itoman,26.12,127.67,301.15,88,75,4.60,JP,1594919558
577,Sturgis,41.80,-85.42,296.15,94,90,2.60,US,1594919475


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [106]:
coordinates = cityDF[['Latitude','Longitude']]
Humidity = cityDF["Humidity"]
figure_layout = {"Weights": Humidity}
fig = gmaps.figure(layout=figure_layout)

In [111]:

heat_layer = gmaps.heatmap_layer(locations, dissipating=False,point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
perfecttemp = cityDF.loc[(cityDF["Max Temp"] >= 294) & (cityDF["Max Temp"] <= 300)]
perfectwind = perfecttemp.loc[(cityDF["Wind Speed"] <= 7)] 
perfectall = perfectwind.loc[(cityDF["Cloudiness"] <= 1)]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
hotel_df = perfectall
hotel_df.reset_index(inplace = True)
hotel_df.drop("index", axis=1,inplace =True)
hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\Kevin\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Chesma,53.81,60.65,295.15,64,0,2.69,RU,1594919620,
1,Havre-St-Pierre,50.23,-63.60,295.15,46,1,5.10,CA,1594919622,
2,Poum,-20.23,164.02,294.10,60,0,4.62,NC,1594919029,
3,Hilo,19.73,-155.09,295.15,78,1,3.10,US,1594919625,
4,Ponta do Sol,32.67,-17.10,296.48,60,1,0.71,PT,1594919627,
5,Mossamedes,-15.20,12.15,295.61,42,0,1.98,AO,1594919634,
6,Westport,41.14,-73.36,298.71,69,1,5.10,US,1594919635,
7,Castro,-24.79,-50.01,294.45,55,0,2.27,BR,1594919571,
8,Abalak,58.13,68.59,297.48,67,0,2.83,RU,1594919642,
9,Saint-Augustin,51.23,-58.65,295.91,38,0,5.17,CA,1594919643,


In [82]:
 params = {"radius": 5000,
           "type": "hotel",
            "key": g_key,
}
for index, row in hotel_df.iterrows():
    Lat = str(row['Latitude'])
    Lon = str(row["Longitude"])
    params["location"] = Lat+","+Lon
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   
    hotel=requests.get(base_url, params=params)
    hotel = hotel.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    except:
        pass
hotel_df

C:\Users\Kevin\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Chesma,53.81,60.65,295.15,64,0,2.69,RU,1594919620,Chesma
1,Havre-St-Pierre,50.23,-63.60,295.15,46,1,5.10,CA,1594919622,Havre-Saint-Pierre
2,Poum,-20.23,164.02,294.10,60,0,4.62,NC,1594919029,Poum
3,Hilo,19.73,-155.09,295.15,78,1,3.10,US,1594919625,Hilo
4,Ponta do Sol,32.67,-17.10,296.48,60,1,0.71,PT,1594919627,Ribeira Brava
5,Mossamedes,-15.20,12.15,295.61,42,0,1.98,AO,1594919634,Namibe
6,Westport,41.14,-73.36,298.71,69,1,5.10,US,1594919635,Norwalk
7,Castro,-24.79,-50.01,294.45,55,0,2.27,BR,1594919571,Castro
8,Abalak,58.13,68.59,297.48,67,0,2.83,RU,1594919642,Abalak
9,Saint-Augustin,51.23,-58.65,295.91,38,0,5.17,CA,1594919643,Saint-Augustin


In [104]:
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Chesma,53.81,60.65,295.15,64,0,2.69,RU,1594919620,Chesma
1,Havre-St-Pierre,50.23,-63.60,295.15,46,1,5.10,CA,1594919622,Havre-Saint-Pierre
2,Poum,-20.23,164.02,294.10,60,0,4.62,NC,1594919029,Poum
3,Hilo,19.73,-155.09,295.15,78,1,3.10,US,1594919625,Hilo
4,Ponta do Sol,32.67,-17.10,296.48,60,1,0.71,PT,1594919627,Ribeira Brava
5,Mossamedes,-15.20,12.15,295.61,42,0,1.98,AO,1594919634,Namibe
6,Westport,41.14,-73.36,298.71,69,1,5.10,US,1594919635,Norwalk
7,Castro,-24.79,-50.01,294.45,55,0,2.27,BR,1594919571,Castro
8,Abalak,58.13,68.59,297.48,67,0,2.83,RU,1594919642,Abalak
9,Saint-Augustin,51.23,-58.65,295.91,38,0,5.17,CA,1594919643,Saint-Augustin


In [105]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
locations

,Latitude,Longitude
0,53.81,60.65
1,50.23,-63.60
2,-20.23,164.02
3,19.73,-155.09
4,32.67,-17.10
5,-15.20,12.15
6,41.14,-73.36
7,-24.79,-50.01
8,58.13,68.59
9,51.23,-58.65


In [112]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker)
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))